In [ ]:
# Install opencv
pip install opencv-python

In [1]:
# Importing modules
import numpy as np
import pandas as pd
import requests
import os
import re
from urllib.parse import urlparse
from pathlib import Path
import os
import cv2
import random
import plotly.express as px
import plotly.graph_objects as go
from PIL import Image, ImageEnhance
import shutil
import tensorflow as tf
from collections import Counter
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import InputLayer, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from scipy.ndimage import rotate, geometric_transform
from tensorflow.image import resize_with_crop_or_pad
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras import regularizers
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Reading the json file retrieved from the github (to be modified to use API)
sets = pd.read_json(r"C:\Users\Jimmy\Desktop\cb-ds-19\00_Projects\ds-final-project-main\en.json")

In [9]:
# Taking a peek inside of the json file
sets.head()

,id,name,series,printedTotal,total,legalities,ptcgoCode,releaseDate,updatedAt,images
0,base1,Base,Base,102,102,{'unlimited': 'Legal'},BS,1999/01/09,2022/10/10 15:12:00,{'symbol': 'https://images.pokemontcg.io/base1...
1,base2,Jungle,Base,64,64,{'unlimited': 'Legal'},JU,1999/06/16,2020/08/14 09:35:00,{'symbol': 'https://images.pokemontcg.io/base2...
2,basep,Wizards Black Star Promos,Base,53,53,{'unlimited': 'Legal'},PR,1999/07/01,2020/08/14 09:35:00,{'symbol': 'https://images.pokemontcg.io/basep...
3,base3,Fossil,Base,62,62,{'unlimited': 'Legal'},FO,1999/10/10,2020/08/14 09:35:00,{'symbol': 'https://images.pokemontcg.io/base3...
4,base4,Base Set 2,Base,130,130,{'unlimited': 'Legal'},B2,2000/02/24,2022/10/10 15:12:00,{'symbol': 'https://images.pokemontcg.io/base4...


In [10]:
# images column contains dictionary. extract those to their own columns (we only care about symbol) and drop images column
images_sets = sets['images'].apply(pd.Series)

sets = pd.concat([sets.drop('images', axis = 1),images_sets], axis = 1)

sets.head()

,id,name,series,printedTotal,total,legalities,ptcgoCode,releaseDate,updatedAt,symbol,logo
0,base1,Base,Base,102,102,{'unlimited': 'Legal'},BS,1999/01/09,2022/10/10 15:12:00,https://images.pokemontcg.io/base1/symbol.png,https://images.pokemontcg.io/base1/logo.png
1,base2,Jungle,Base,64,64,{'unlimited': 'Legal'},JU,1999/06/16,2020/08/14 09:35:00,https://images.pokemontcg.io/base2/symbol.png,https://images.pokemontcg.io/base2/logo.png
2,basep,Wizards Black Star Promos,Base,53,53,{'unlimited': 'Legal'},PR,1999/07/01,2020/08/14 09:35:00,https://images.pokemontcg.io/basep/symbol.png,https://images.pokemontcg.io/basep/logo.png
3,base3,Fossil,Base,62,62,{'unlimited': 'Legal'},FO,1999/10/10,2020/08/14 09:35:00,https://images.pokemontcg.io/base3/symbol.png,https://images.pokemontcg.io/base3/logo.png
4,base4,Base Set 2,Base,130,130,{'unlimited': 'Legal'},B2,2000/02/24,2022/10/10 15:12:00,https://images.pokemontcg.io/base4/symbol.png,https://images.pokemontcg.io/base4/logo.png


In [14]:
# Download all of the logo images as train001.jpg in a folder named after the set (using name column from sets), which is in term inside of a folder named after series

# Define base path location of all the images
base_path = "../ds-final-project-main/PokemonTCG/Set_Symbol/"

# Function to sanitize folder names
def sanitize_folder_name(name):
    # Replace invalid characters with an underscore or remove them
    return re.sub(r'[<>:"/\\|?*]', '', name)

# Function to download and save an image as train.jpg in the specified folder
def download_image(url, series, name, base_path):
    # Sanitize the series and name to create valid folder names
    series_safe = sanitize_folder_name(series)
    name_safe = sanitize_folder_name(name)
    
    # Construct the full path using base_path, sanitized series, and name
    final_folder_path = os.path.join(base_path, series_safe, name_safe)
    
    # Ensure the target folder exists
    os.makedirs(final_folder_path, exist_ok=True)
    
    # Full path to save the image as train.jpg
    file_path = os.path.join(final_folder_path, '001.jpg')
    
    try:
        # Send a GET request to fetch the image
        response = requests.get(url)
        response.raise_for_status()
        
        # Save the image to the specified path
        with open(file_path, 'wb') as f:
            f.write(response.content)
        
        print(f"Downloaded 001.jpg to {final_folder_path}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download train.jpg for {series_safe}/{name_safe}: {e}")

# Loop through the DataFrame and download images
for index, row in sets.iterrows():
    download_image(row['symbol'], row['series'], row['name'], base_path)

Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Jungle
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Wizards Black Star Promos
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Fossil
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base Set 2
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Team Rocket
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Gym\Gym Heroes
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Gym\Gym Challenge
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Neo\Neo Genesis
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Neo\Neo Discovery
Downloaded train.jpg to ../ds-final-project-main/PokemonTCG/Set_Symbol/Other\Southern Islands
Downloaded train.jpg to ../ds-f

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from PIL import Image
import random
import plotly.graph_objects as go
import os

# Custom function to apply a random inversion of black and white colors
def random_invert_black_and_white(image, invert_prob=0.5):
    if random.random() < invert_prob:
        grayscale_image = np.mean(image, axis=-1)
        inverted_image = 255 - grayscale_image
        return np.stack([inverted_image] * 3, axis=-1).astype('uint8')
    else:
        return image.astype('uint8')

# Custom function to rotate and keep the image within its bounds
def rotate_and_keep_shape(image, angle):
    pil_image = Image.fromarray(image.astype('uint8'))
    rotated_image = pil_image.rotate(angle, resample=Image.BICUBIC, fillcolor=(0, 0, 0))
    return np.array(rotated_image).astype('uint8')

# Custom function to zoom and pad
def zoom_and_pad(image, zoom_factor):
    pil_image = Image.fromarray(image.astype('uint8'))
    width, height = pil_image.size
    
    # Calculate new dimensions
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)
    
    # Resize to the new dimensions
    zoomed_image = pil_image.resize((new_width, new_height), resample=Image.BICUBIC)
    
    # Center crop to original size
    left = (new_width - width) / 2
    top = (new_height - height) / 2
    right = (new_width + width) / 2
    bottom = (new_height + height) / 2
    
    zoomed_image = zoomed_image.crop((left, top, right, bottom))
    
    return np.array(zoomed_image).astype('uint8')

# Custom preprocessing function that combines all augmentations
def custom_augmentation(image):
    # Apply random inversion
    image = random_invert_black_and_white(image, invert_prob=0.5)
    # Apply random rotation
    rotation_angle = random.uniform(-180, 180)
    image = rotate_and_keep_shape(image, rotation_angle)
    # Apply random zoom
    zoom_factor = random.uniform(0.8, 1.2)
    image = zoom_and_pad(image, zoom_factor)
    
    # Ensure the image has an alpha channel
    if image.shape[-1] == 3:
        alpha_channel = np.ones((image.shape[0], image.shape[1]), dtype='uint8') * 255
        image = np.dstack((image, alpha_channel))
    return image

# Load and rename the original image
original_image_path = "../ds-final-project-main/PokemonTCG/Set_Symbol/Scarlet & Violet/Twilight Masquerade/train001.jpg"
renamed_image_path = "../ds-final-project-main/PokemonTCG/Set_Symbol/Scarlet & Violet/Twilight Masquerade/train001.png"

# Open the original image and save it with the new name
original_image = Image.open(original_image_path).convert('RGBA')
original_image.save(renamed_image_path, format='PNG')

# Reload the renamed image
image = load_img(renamed_image_path, color_mode='rgba')
image_array = img_to_array(image)
image_array = image_array.reshape((1,) + image_array.shape)

# Create an ImageDataGenerator instance with standard augmentations
datagen = ImageDataGenerator(
    shear_range=0.5,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest',
    preprocessing_function=custom_augmentation

# Generate augmented images and save/display them
iterator = datagen.flow(image_array, batch_size=1)

# Directory to save augmented images
save_directory = "../ds-final-project-main/PokemonTCG/Set_Symbol/Scarlet & Violet/Twilight Masquerade"

# Ensure the directory exists
os.makedirs(save_directory, exist_ok=True)

# Generate and save 99 augmented images
for i in range(2, 101):
    try:
        batch = next(iterator)
        augmented_image = batch[0].astype('uint8')
        
        # Ensure the image is in RGBA format
        if augmented_image.shape[-1] == 3:
            alpha_channel = np.ones((augmented_image.shape[0], augmented_image.shape[1]), dtype='uint8') * 255
            augmented_image = np.dstack((augmented_image, alpha_channel))
        
        # Save the image with leading zeros in the filename
        save_path = f"{save_directory}/train{i:03}.png"
        Image.fromarray(augmented_image, 'RGBA').save(save_path, format='PNG')
        
        print(f"Saved image to {save_path}")
        
    except Exception as e:
        print(f"Error processing image {i}: {e}")


In [19]:
# Only 1 example of each symbol is available. In order to train, we are bulking out our training set (and creating images that will be used for test/valid set) by applying our data augmentation to the image and exporting them as images.
# Custom functions for performing data augmentation. Early testing showed that extreme data augmentation shifted the images outside of the usable image space
def random_invert_black_and_white(image, invert_prob=0.5):
    if random.random() < invert_prob:
        grayscale_image = np.mean(image, axis=-1)
        inverted_image = 255 - grayscale_image
        return np.stack([inverted_image] * 3, axis=-1).astype('uint8')
    else:
        return image.astype('uint8')

def adjust_brightness_contrast_saturation(image, brightness=0.1, contrast=0.1, saturation=0.1):
    pil_image = Image.fromarray(image.astype('uint8')).convert('RGB')
    
    # Apply random brightness
    enhancer = ImageEnhance.Brightness(pil_image)
    pil_image = enhancer.enhance(1 + random.uniform(-brightness, brightness))
    
    # Apply random contrast
    enhancer = ImageEnhance.Contrast(pil_image)
    pil_image = enhancer.enhance(1 + random.uniform(-contrast, contrast))
    
    # Apply random saturation
    enhancer = ImageEnhance.Color(pil_image)
    pil_image = enhancer.enhance(1 + random.uniform(-saturation, saturation))
    
    return np.array(pil_image).astype('uint8')

def add_gaussian_noise(image, mean=0, std=25):
    noise = np.random.normal(mean, std, image.shape)
    noisy_image = image + noise
    return np.clip(noisy_image, 0, 255).astype('uint8')

def apply_blur(image, kernel_size=3):
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    return blurred_image.astype('uint8')

def apply_shiny_overlay(image, alpha=0.3):
    overlay = np.ones_like(image) * 255  # White overlay
    shiny_image = cv2.addWeighted(image.astype('uint8'), 1 - alpha, overlay.astype('uint8'), alpha, 0)
    return shiny_image.astype('uint8')

def add_random_shadow(image, shadow_intensity=0.5):
    shadow_mask = np.random.uniform(0.3, 1.0, size=(image.shape[0], image.shape[1], 1))
    shadow_mask = np.repeat(shadow_mask, 3, axis=-1)
    shadow_image = np.clip(image * shadow_mask, 0, 255).astype('uint8')
    return shadow_image

def rotate_and_keep_shape(image, angle):
    pil_image = Image.fromarray(image.astype('uint8'))
    rotated_image = pil_image.rotate(angle, resample=Image.BICUBIC, fillcolor=(0, 0, 0))
    return np.array(rotated_image).astype('uint8')

def ensure_consistent_dimensions(image, target_height, target_width):
    pil_image = Image.fromarray(image)
    pil_image = pil_image.resize((target_width, target_height), Image.BICUBIC)
    return np.array(pil_image).astype('uint8')

def zoom_and_pad(image, zoom_factor):
    pil_image = Image.fromarray(image.astype('uint8'))
    width, height = pil_image.size
    
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)
    
    zoomed_image = pil_image.resize((new_width, new_height), resample=Image.BICUBIC)
    
    left = (new_width - width) / 2
    top = (new_height - height) / 2
    right = (new_width + width) / 2
    bottom = (new_height + height) / 2
    
    zoomed_image = zoomed_image.crop((left, top, right, bottom))
    
    # Ensure consistent dimensions
    target_height, target_width = height, width
    return ensure_consistent_dimensions(np.array(zoomed_image), target_height, target_width)

def custom_augmentation(image):

    if random.random() < 0.3:
        image = random_invert_black_and_white(image, invert_prob=0.5)
    if random.random() < 0.4:
        image = adjust_brightness_contrast_saturation(image, brightness=0.2, contrast=0.2, saturation=0.2)
    if random.random() < 0.4:
        image = add_gaussian_noise(image, mean=0, std=15)
    if random.random() < 0.3:
        image = apply_blur(image, kernel_size=3)
    if random.random() < 0.3:
        image = apply_shiny_overlay(image, alpha=0.2)
    if random.random() < 0.4:
        image = add_random_shadow(image, shadow_intensity=0.5)
    if random.random() < 0.9:
        rotation_angle = random.uniform(-180, 180)
        image = rotate_and_keep_shape(image, rotation_angle)
    if random.random() < 0.8:
        zoom_factor = random.uniform(0.4, 1.2)
        image = zoom_and_pad(image, zoom_factor)
    return image

def process_image(image_path, save_directory):
    try:
        # Prepare ImageDataGenerator
        image = load_img(image_path)
        image_array = img_to_array(image)
        image_array = image_array.reshape((1,) + image_array.shape)

        datagen = ImageDataGenerator(
            shear_range=0.5,
            horizontal_flip=False,
            vertical_flip=False,
            fill_mode='nearest',
            preprocessing_function=custom_augmentation
        )

        iterator = datagen.flow(image_array, batch_size=1)

        # Generate and save 199 augmented images
        for i in range(2, 201):
            try:
                batch = next(iterator)
                augmented_image = batch[0].astype('uint8')
                
                # Ensure the image is in RGBA format
                if augmented_image.shape[-1] == 3:
                    alpha_channel = np.ones((augmented_image.shape[0], augmented_image.shape[1]), dtype='uint8') * 255
                    augmented_image = np.dstack((augmented_image, alpha_channel))
                
                # Save the image with leading zeros in the filename
                save_path = os.path.join(save_directory, f"{i:03}.png")
                Image.fromarray(augmented_image, 'RGBA').save(save_path, format='PNG')
                
                print(f"Saved image to {save_path}")
                
            except Exception as e:
                print(f"Error processing image {i}: {e}")

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

# Base directory path
base_directory = "../ds-final-project-main/PokemonTCG/Set_Symbol/"

# Walk through the directory tree
for root, dirs, files in os.walk(base_directory):
    for file in files:
        if file.lower() == "001.jpg":
            image_path = os.path.join(root, file)
            save_directory = root
            print(f"Processing {image_path}")
            process_image(image_path, save_directory)

Processing ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train001.jpg
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train002.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train003.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train004.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train005.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train006.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train007.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train008.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train009.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train010.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base\train011.png
Saved image to ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Ba

In [6]:
# Originally downloaded 001.jpg should be converted to RGBA and converted to .png filetype

# Function to convert .jpg to .png with RGBA format
def convert_jpg_to_png_with_rgba(jpg_path, png_path):
    with Image.open(jpg_path) as img:
        rgba_img = img.convert('RGBA')
        rgba_img.save(png_path)

In [11]:
# Create train/valid/test sets
# Base directory path
base_directory = "../ds-final-project-main/PokemonTCG/Set_Symbol/"
# Function to convert .jpg to .png with RGBA format
def convert_jpg_to_png_with_rgba(jpg_path, png_path):
    with Image.open(jpg_path) as img:
        rgba_img = img.convert('RGBA')
        rgba_img.save(png_path)

# Function to clear existing files in a folder
def clear_folder(folder_path):
    if os.path.exists(folder_path):
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)

# Create subfolders and distribute images
for series in os.listdir(base_directory):
    series_path = os.path.join(base_directory, series)
    
    if not os.path.isdir(series_path):
        continue

    for set_folder in os.listdir(series_path):
        set_path = os.path.join(series_path, set_folder)
        
        if not os.path.isdir(set_path):
            continue
        
        print(f"Processing folder: {set_path}")
        
        # Create subfolders: train, test, and valid
        train_path = os.path.join(set_path, 'train')
        test_path = os.path.join(set_path, 'test')
        valid_path = os.path.join(set_path, 'valid')
        
        # Clear existing files in the subfolders
        clear_folder(train_path)
        clear_folder(test_path)
        clear_folder(valid_path)
        
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)
        os.makedirs(valid_path, exist_ok=True)
        
        # Collect .jpg and .png images
        images = set()  # Use a set to avoid duplicates
        for file in os.listdir(set_path):
            file_path = os.path.join(set_path, file)
            if file.lower().endswith('.jpg'):
                png_filename = f"{os.path.splitext(file)[0]}.png"
                png_path = os.path.join(set_path, png_filename)
                if not os.path.exists(png_path):
                    convert_jpg_to_png_with_rgba(file_path, png_path)
                images.add(png_path)
                os.remove(file_path)
            elif file.lower().endswith('.png'):
                images.add(file_path)
        
        # Convert set to list
        images = list(images)
        
        # Debugging: Check the number of images found
        num_images = len(images)
        print(f"Found {num_images} images in {set_path}.")
        
        if num_images < 20:
            print(f"Warning: Not enough images in {set_path}. Found {num_images} images. Skipping.")
            continue
        
        # Shuffle images randomly
        random.shuffle(images)

        # Calculate split sizes
        num_train = int(0.45 * num_images)
        num_valid = int(0.45 * num_images)
        num_test = num_images - num_train - num_valid
        
        # Ensure num_test is exactly 20
        num_test = min(num_test, 20)
        
        # Select images for train, valid, and test
        train_images = images[:num_train]
        valid_images = images[num_train:num_train + num_valid]
        test_images = images[num_train + num_valid:num_train + num_valid + num_test]
        
        # Ensure correct number of images in each category
        if len(train_images) != num_train or len(valid_images) != num_valid or len(test_images) != num_test:
            print(f"Error in splitting images for {set_path}.")
            continue
        
        # Copy images to their respective subfolders with new names
        for idx, img_path in enumerate(train_images):
            new_filename = f"train{str(idx + 1).zfill(3)}.png"
            shutil.copy(img_path, os.path.join(train_path, new_filename))
        
        for idx, img_path in enumerate(valid_images):
            new_filename = f"valid{str(idx + 1).zfill(3)}.png"
            shutil.copy(img_path, os.path.join(valid_path, new_filename))
        
        for idx, img_path in enumerate(test_images):
            new_filename = f"test{str(idx + 1).zfill(3)}.png"
            shutil.copy(img_path, os.path.join(test_path, new_filename))

print("Images have been successfully converted, copied, and renamed into train, test, and valid subfolders.")

Processing folder: ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base
Found 200 images in ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base.
Processing folder: ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base Set 2
Found 200 images in ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Base Set 2.
Processing folder: ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Fossil
Found 200 images in ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Fossil.
Processing folder: ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Jungle
Found 200 images in ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Jungle.
Processing folder: ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Team Rocket
Found 200 images in ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Team Rocket.
Processing folder: ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Wizards Black Star Promos
Found 200 images in ../ds-final-project-main/PokemonTCG/Set_Symbol/Base\Wizards Black Star Promos.


In [ ]:
# Define the ImageDataGenerator with augmentation options
datagen = ImageDataGenerator(
    rescale=1./255
)

# Create the data generator
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical'
)

# Use train_generator for model training

In [ ]:
def apply_blur(image, kernel_size=5):
    """Apply blur to the image using TensorFlow."""
    blur = tf.image.adjust_brightness(image, delta=0.0)
    return blur

def blur_layer(kernel_size=5):
    """Create a blur layer using a Lambda layer."""
    return tf.keras.layers.Lambda(lambda x: apply_blur(x, kernel_size))

# Define the input shape
input_shape = (128, 128, 3)

# Build the model with blur augmentation
model = tf.keras.Sequential([
    # Input layer specifying the input shape
    tf.keras.layers.InputLayer(input_shape=input_shape),

    # Data augmentation layers
    blur_layer(kernel_size=5),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),

    # Model layers
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()